In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

PART 1:

In [69]:
df = pd.read_csv('/workspaces/Marquette_teaching_finance/used_cars.csv')

In [70]:
df[['price']].head()

,price
0,"$10,300"
1,"$38,005"
2,"$54,598"
3,"$15,500"
4,"$34,999"


In [71]:
print("Initial data shape:", df.shape)

Initial data shape: (4009, 12)


In [72]:
print("Columns in dataset:", df.columns.tolist())

Columns in dataset: ['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title', 'price']


In [73]:
if 'milage' in df.columns:
    df.rename(columns={'milage': 'mileage'}, inplace=True)


In [74]:
for col in ['mileage', 'model_year', 'price']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [75]:
df[['price']].head()

,price
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [61]:
for col in ['mileage', 'model_year', 'price']:
    if col in df.columns and df[col].isnull().any():
        if col in ['mileage', 'price']:
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)

/home/codespace/.python/current/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/tmp/ipykernel_32261/2427229060.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/home/codespace/.python/current/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/tmp/ipykernel_32

In [62]:
categorical_cols = ['fuel_type', 'clean_title', 'engine', 'transmission', 'ext_col', 'int_col']

In [63]:
categorical_cols = ['fuel_type', 'clean_title', 'engine', 'transmission', 'ext_col', 'int_col']
existing_categorical = [col for col in categorical_cols if col in df.columns]
if existing_categorical:
    df = pd.get_dummies(df, columns=existing_categorical, drop_first=True)
print("Final DataFrame columns after preprocessing:", df.columns.tolist())

Final DataFrame columns after preprocessing: ['brand', 'model', 'model_year', 'mileage', 'accident', 'price', 'fuel_type_E85 Flex Fuel', 'fuel_type_Gasoline', 'fuel_type_Hybrid', 'fuel_type_Plug-In Hybrid', 'fuel_type_not supported', 'fuel_type_–', 'engine_1.3L I3 12V GDI DOHC Turbo', 'engine_1.3L I3 12V MPFI DOHC Turbo', 'engine_1.4L I4 16V GDI DOHC Turbo', 'engine_1.5 Liter Turbo', 'engine_1.5L I3 12V GDI DOHC Turbo', 'engine_1.5L I3 12V PDI DOHC Turbo', 'engine_1.5L I4 16V GDI DOHC Turbo', 'engine_1.6L I-4 gasoline direct injection, DOHC, variable valve control', 'engine_1.6L I4 16V GDI DOHC', 'engine_1.6L I4 16V GDI DOHC Hybrid', 'engine_1.6L I4 16V GDI DOHC Turbo', 'engine_1.6L I4 16V GDI DOHC Turbo Hybrid', 'engine_1.6L I4 16V MPFI DOHC', 'engine_1.8 Liter', 'engine_101.0HP 1.4L 4 Cylinder Engine Gasoline Fuel', 'engine_1020.0HP Electric Motor Electric Fuel System', 'engine_104.0HP 1.6L 4 Cylinder Engine Gasoline Fuel', 'engine_106.0HP 1.5L 4 Cylinder Engine Gasoline Fuel', 'engi

In [64]:
df.shape

(4009, 1691)

In [65]:
df.columns

Index(['brand', 'model', 'model_year', 'mileage', 'accident', 'price',
       'fuel_type_E85 Flex Fuel', 'fuel_type_Gasoline', 'fuel_type_Hybrid',
       'fuel_type_Plug-In Hybrid',
       ...
       'int_col_Titan Black / Quarzit', 'int_col_Tupelo',
       'int_col_Very Light Cashmere', 'int_col_WHITE', 'int_col_Walnut',
       'int_col_Whisper Beige', 'int_col_White', 'int_col_White / Brown',
       'int_col_Yellow', 'int_col_–'],
      dtype='object', length=1691)

In [66]:
df[['price']].head()

,price
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [ ]:
# df = pd.get_dummies(df, columns=existing_categorical, drop_first=True)


In [ ]:
if df['price'].notnull().sum() > 0:
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    # Use a standard multiplier of 1.5
    multiplier = 1.5  
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    df_filtered = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
    print(df_filtered.shape)
    if df_filtered.empty:
        print("DataFrame empty after outlier removal with multiplier=1.5. Trying multiplier=3.0.")
        multiplier = 3.0
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        df_filtered = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
        if df_filtered.empty:
            print("Warning: DataFrame still empty after outlier removal. Skipping outlier removal.")
        else:
            df = df_filtered
    else:
        df = df_filtered
else:
    print("Warning: 'price' column has no valid numbers.")

In [ ]:
df.dropna(inplace=True)
print("Data shape after preprocessing:", df.shape)


PART 2:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
if 'price' not in df.columns:
    raise ValueError("CSV file does not contain a 'price' column.")

In [ ]:
X = df.drop('price', axis=1)
y = df['price']

In [ ]:
if X.empty or X.shape[0] == 0:
    print("Warning: Feature set X is empty after preprocessing. Creating dummy data for modeling.")
    dummy_size = 100  # You can adjust the number of dummy samples as needed.
    X = pd.DataFrame({'dummy': np.linspace(0, 100, dummy_size)})
    y = pd.Series(np.linspace(1000, 5000, dummy_size))


In [ ]:
print("Final feature set columns:", X.columns.tolist())
print("Feature set shape:", X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set shape:", X_train.shape)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [ ]:
y_pred = lr_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Linear Regression RMSE:", rmse)

In [ ]:
X_test

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_test.iloc[:, 0], y_test, color='blue', label='Actual Prices', alpha=0.6)
plt.scatter(X_test.iloc[:, 0], y_pred, color='red', label='Predicted Prices', alpha=0.6)
plt.xlabel(X_test.columns[0])
plt.ylabel('Price')
plt.title('Linear Regression: Actual vs. Predicted Prices')
plt.legend()
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
k = 5
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train, y_train)

In [ ]:
y_pred_knn = knn_model.predict(X_test)


In [ ]:
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
print("KNN Regression RMSE (k={}): {:.2f}".format(k, rmse_knn))

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_test.iloc[:, 0], y_test, color='blue', label='Actual Prices', alpha=0.6)
plt.scatter(X_test.iloc[:, 0], y_pred_knn, color='green', label='KNN Predicted Prices', alpha=0.6)
plt.xlabel(X_test.columns[0])
plt.ylabel('Price')
plt.title('KNN Regression (k={}) - Actual vs. Predicted Prices'.format(k))
plt.legend()
plt.show()